In [2]:
import matplotlib
matplotlib.use('TkAgg')
import numpy as np
import matplotlib.pyplot as plt
import cv2
import skimage

In [3]:
import math

In [ ]:
plt.figure(figsize=(20,100))
plt.subplot(151)
plt.imshow(cv2.cvtColor(cv2.imread("cmpe-building/left-2.jpg"),cv2.COLOR_BGR2RGB))

plt.axis('off')
plt.subplot(152)
plt.imshow(cv2.cvtColor(cv2.imread("cmpe-building/left-1.jpg"),cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.subplot(153)
plt.imshow(cv2.cvtColor(cv2.imread("cmpe-building/middle.jpg"),cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.subplot(154)
plt.imshow(cv2.cvtColor(cv2.imread("cmpe-building/right-1.jpg"),cv2.COLOR_BGR2RGB))
plt.axis('off')            
plt.subplot(155)
plt.imshow(cv2.cvtColor(cv2.imread("cmpe-building/right-2.jpg"),cv2.COLOR_BGR2RGB))
plt.axis('off')            
plt.show()


In [4]:
def computeH(points_left,points_right):
    p1=points_left
    p2=points_right
    A = []
    for i in range(0, len(p1)):
        x, y = p1[i][0], p1[i][1]
        u, v = p2[i][0], p2[i][1]
        A.append([x, y, 1, 0, 0, 0, -u*x, -u*y, -u])
        A.append([0, 0, 0, x, y, 1, -v*x, -v*y, -v])
    A = np.asarray(A)
    U, S, Vh = np.linalg.svd(A)
    L = Vh[-1,:] / Vh[-1,-1]
    return L.reshape(3, 3)

def center_and_normalize_points(points):
    centroid = np.mean(points, axis=0)
    rms = math.sqrt(np.sum((points - centroid) ** 2) / points.shape[0])
    norm_factor = math.sqrt(2) / rms
    matrix = np.array([[norm_factor, 0, -norm_factor * centroid[0]],
                       [0, norm_factor, -norm_factor * centroid[1]],
                       [0, 0, 1]])
    pointsh = np.row_stack([points.T, np.ones((points.shape[0]),)])
    new_pointsh = (matrix @ pointsh).T
    new_points = new_pointsh[:, :2]
    new_points[:, 0] /= new_pointsh[:, 2]
    new_points[:, 1] /= new_pointsh[:, 2]

    return matrix, new_points
def compute_H_with_normalization(points_left,points_right):
    t1,new_points_left=center_and_normalize_points(points_left)
    t2,new_points_right=center_and_normalize_points(points_right)
    H=computeH(np.array(new_points_left),np.array(new_points_right))
    return np.dot(np.dot(np.linalg.inv(t2),H),t1)


In [7]:
def warp(image,H):
    im=image
    # This part will will calculate the X and Y offsets
    bunchX=[]; bunchY=[]

    tt = np.array([[1],[1],[1]])
    tmp = np.dot(H,tt)
    bunchX.append(tmp[0]/tmp[2])
    bunchY.append(tmp[1]/tmp[2])

    tt = np.array([[im.shape[1]],[1],[1]])
    tmp = np.dot(H,tt)
    bunchX.append(tmp[0]/tmp[2])
    bunchY.append(tmp[1]/tmp[2])

    tt = np.array([[1],[im.shape[0]],[1]])
    tmp = np.dot(H,tt)
    bunchX.append(tmp[0]/tmp[2])
    bunchY.append(tmp[1]/tmp[2])

    tt = np.array([[im.shape[1]],[im.shape[0]],[1]])
    tmp = np.dot(H,tt)
    bunchX.append(tmp[0]/tmp[2])
    bunchY.append(tmp[1]/tmp[2])

    refX1 = int(np.min(bunchX))
    refX2 = int(np.max(bunchX))
    refY1 = int(np.min(bunchY))
    refY2 = int(np.max(bunchY))

    # Final image whose size is defined by the offsets previously calculated
    final = np.zeros((int(refY2-refY1),int(refX2-refX1),3))
    # final = np.zeros((2000,2000,3))

    # Iterate over the imagine to transform every pixel
    for i in range(im.shape[0]):
            for j in range(im.shape[1]):

                    tt = np.array([[j],[i],[1]])
                    tmp = np.dot(H,tt)
                    x1=int(tmp[0]/tmp[2])-refX1
                    y1=int(tmp[1]/tmp[2])-refY1

                    if x1>0 and y1>0 and y1<refY2-refY1 and x1<refX2-refX1:
                            final[y1,x1,:]=im[i,j,:]
                            
    return final.astype(np.uint8)
    # Hi = np.linalg.inv(H)
    # for i in range(final.shape[0]):
    #         for j in range(final.shape[1]):
    #                 if sum(final[i,j,:])==0:
    #                         tt = np.array([[j+refX1],[i+refY1],[1]])
    #                         tmp = np.dot(Hi,tt)
    #                         x1=int(tmp[0]/tmp[2])
    #                         y1=int(tmp[1]/tmp[2])

    #                         if x1>0 and y1>0 and x1<im.shape[1] and y1<im.shape[0]:
    #                                 final[i,j,:] = im[y1,x1,:]


In [47]:
def blend(image_left,final):
    final_res_img=np.zeros((max(image_left.shape[0],final.shape[0]),max(image_left.shape[1],final.shape[1]),3),dtype=np.uint8)
    for j in range(final_res_img.shape[0]):
        for i in range(final_res_img.shape[1]):
            if j<image_left.shape[0] and i<image_left.shape[1]:
                if np.sum(image_left[j,i,:])>np.sum(final[j,i,:]):                                             
                    final_res_img[j,i]=image_left[j,i]
                else:
                    final_res_img[j,i]=final[j,i]
    return final_res_img
def get_common_points(left_img_path,right_img_path):
    plt.figure(figsize=(20,20))
    image_left=cv2.imread(left_img_path)
    plt.imshow(image_left)
    points_left = np.array(plt.ginput(4))
    
    image_right=cv2.imread(right_img_path)
    plt.figure(figsize=(20,20))
    plt.imshow(image_right)
    points_right = np.array(plt.ginput(4))
    
    print("Selected points for left img:" ,points_left)
    print("Selected points for right img:" ,points_right)
    return (points_left,points_right),(image_left,image_right)

# (points_left_2_middle,points_middle_left_2),(image_left_2,image_middle)=get_common_points("cmpe-building/left-2.jpg","cmpe-building/middle.jpg")
# H_left_2_middle=compute_H_with_normalization(points_left_2_middle,points_middle_left_2)
# warped_left=warp(image_left_2,np.linalg.inv(H_left_2_middle))
# cv2.imshow("left_middle",warped_left)
cv2.imshow("left_blend",blend(warped_left,image_middle))
# (points_middle_right_2,points_right_2_middle),(image_middle,image_right_2)=get_common_points("cmpe-building/middle.jpg","cmpe-building/right-2.jpg")
# H_middle_2_right=compute_H_with_normalization(points_middle_right_2,points_right_2_middle)
# warped_right=warp(image_right_2,H_middle_2_right)
# cv2.imshow("right_middle",blend(warp(image_left_2,np.linalg.inv(H_left_2_middle)),blend(image_middle,warped_right)))



IndexError: index 1280 is out of bounds for axis 1 with size 1280

In [41]:
zaa==None

True

In [33]:
cv2.imshow("aaa",cv2.warpPerspective(image_right_2,cv2.findHomography(points_middle_right_2,points_right_2_middle)[0],(image_right_2.shape[0],image_right_2.shape[1])))

In [ ]:
[[459.39932753 787.174096  ]
 [432.76472104 747.85634355]
 [715.59887569 609.61005271]
 [910.9193233  534.77949162]
 [964.18853629 396.53320078]]
[[209.54135234 808.73544411]
 [175.29685828 764.34443329]
 [498.71707997 610.87836731]
 [619.20696648 575.36555865]
 [730.81865083 399.06982997]]

In [45]:
warped_right.shape

(1401, 1869, 3)

In [16]:
imae=blend(image_middle,image_left_2)

In [9]:
cv2.imshow("aa",warped_right)

In [20]:
warped_left.shape

(712, 1486, 3)

In [ ]:
H=np.dot(np.dot(np.linalg.inv(t2),H),t1)
skimage.__g

In [7]:
# points_left=np.load("left-1.npz.npy")
# points_right=np.load("left-2.npz.npy")
points_left=np.array([[ 970.44746163,  396],
 [ 998.78276269  ,397],
 [1071.98229044  ,400],
 [1220.74262102  ,396],
 [ 999.96340024  ,417]])
points_right=np.array([[733.13931523 ,396],
 [760.29397875 ,397],
 [831.1322314  ,400],
 [951.55726092 ,396],
 [762.65525384 ,417]])

In [14]:
cv2.imshow("a",cv2.warpPerspective(image_right_2,np.linalg.inv(H_middle_2_right),(image_right_2.shape[0],image_right_2.shape[1])))

In [21]:
H=find_H(points_left,points_right)

In [30]:
transform.warp(image_right,H).shape

(960, 1280, 3)

In [5]:
H

array([[-4.70000518e-01, -7.63031937e-01,  7.90669984e+02],
       [-2.70738276e-01, -4.83146824e-01,  4.64575377e+02],
       [-5.74609589e-04, -1.01495730e-03,  1.00000000e+00]])

In [43]:
estimate(points_left,points_right)

array([[ 1.08936590e+00, -9.69937909e-02, -2.44543147e+02],
       [ 9.98341117e-02,  9.70490116e-01, -6.95126462e+01],
       [ 1.93035791e-04, -1.08655541e-04,  9.07011461e-01]])

In [119]:
t1,new_points_left=center_and_normalize_points(points_left)
t2,new_points_right=center_and_normalize_points(points_right)
H=find_H(np.array(new_points_left),np.array(new_points_right))
H

array([[ 0.93890924, -0.05972727,  0.05940537],
       [-0.01458205,  1.00524821, -0.04512195],
       [ 0.02698676, -0.0296329 ,  1.        ]])

In [52]:
np.dot(np.dot(np.linalg.inv(t2),H),t1)

array([[ 1.08936590e+00, -9.69937909e-02, -2.44543147e+02],
       [ 9.98341117e-02,  9.70490116e-01, -6.95126462e+01],
       [ 1.93035791e-04, -1.08655541e-04,  9.07011461e-01]])

In [33]:
find_H(points_left,points_right)

array([[ 5.44141392e-01, -1.11760633e-01,  3.71465192e+02],
       [-1.36847154e-01,  7.20698807e-01,  1.12529631e+02],
       [-2.78796720e-04, -1.16526305e-04,  1.00000000e+00]])

In [32]:
new_points_left

array([[-1.60539512,  1.11159088],
       [-1.77542814,  0.91321902],
       [-0.38115736,  0.255758  ],
       [-0.04109131,  0.28409684],
       [ 0.29330696,  0.12539935],
       [ 1.27383073, -0.22600223],
       [ 0.8090738 , -0.78144343],
       [ 1.42686045, -1.68261845]])

In [ ]:
def computeH(im1Points,im2Points):
    pass

In [ ]:
import numpy as np
np.array(x).T

In [60]:
H=cv2.findHomography(points_left,points_right)

ERROR! Session/line number was not unique in database. History logging moved to new session 1339


In [45]:
points_left.T

array([[ 970.44746163,  998.78276269, 1071.98229044, 1220.74262102,
         999.96340024],
       [ 396.52479339,  397.70543093,  400.06670602,  396.52479339,
         416.59563164]])

In [137]:
h_cv2,status=cv2.findHomography(points_right,points_left)

In [54]:
h_cv2

array([[ 6.73699075e-02, -9.97035318e-01,  7.17172924e+02],
       [-1.19999237e-01, -4.40606045e-02,  2.56997622e+02],
       [-3.02755847e-04, -9.98287838e-04,  1.00000000e+00]])

In [ ]:
points_right[1:].dtype

In [6]:
np.float32([[56,65],[368,52],[28,387],[389,390]]).shape
p_l=points_left[1:].astype(np.float32)
p_r=points_right[1:].astype(np.float32)

In [35]:
M = cv2.getPerspectiveTransform(p_r,p_l)


In [41]:
H

array([[-2.22620571e-01, -6.24834723e-01,  5.88067189e+02],
       [-2.90438308e-01, -5.85192914e-01,  6.20106084e+02],
       [-4.36254493e-04, -9.80883972e-04,  1.00000000e+00]])

In [130]:
# Iterate over the imagine to transform every pixel
H=np.dot(np.dot(np.linalg.inv(t2),H),t1)
image=image_right.astype(np.float64)
image /= 255

final=np.zeros(shape=(image_right.shape[0]*2,image_right.shape[1]*2,3))
for i in range(image_right.shape[0]):
        for j in range(image_right.shape[1]):
#                 z_ = H[6] * x + H[7] * y + H[8]
#                 x_[0] = (H[0] * x + H[1] * y + H[2]) / z_
#                 y_[0] = (H[3] * x + H[4] * y + H[5]) / z_
                tt = np.array([[j],[i],[1]])
                tmp = np.dot(H,tt)
                x1=int(tmp[0]/tmp[2])
                y1=int(tmp[1]/tmp[2])
            
                final[y1][x1]=image[i,j,:]
min_val = final.min()
max_val = final.max()
final=np.clip(final, min_val, max_val)

In [56]:
np.where

array([1, 4, 4, ..., 1, 1, 1], dtype=int64)

In [131]:
cv2.imshow("Warp",final)

In [14]:
image_right.shape*2

(960, 1280, 3, 960, 1280, 3)

In [30]:
H

array([[ 1.70939024e-01, -1.74395501e+00,  6.27002975e+02],
       [ 2.05106418e-02, -8.19396436e-01,  3.59862907e+02],
       [ 5.18263019e-05, -2.30003005e-03,  1.00000000e+00]])

In [31]:
cv2.imshow("warp",transform.warp(image_right,H))

NameError: name 'transform' is not defined

In [106]:
imageA=image_right
imageB=image_left
b=H.copy()


result = cv2.warpPerspective(imageB, H,(imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB

In [107]:
plt.figure(figsize=(20,20))
plt.imshow(result)
plt.show()

D:\Development\Continuum\anaconda3\lib\site-packages\matplotlib\pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\Development\Continuum\anaconda3\lib\tkinter\__init__.py", line 1699, in __call__
    return self.func(*args)
  File "D:\Development\Continuum\anaconda3\lib\site-packages\matplotlib\backends\_backend_tk.py", line 227, in resize
    self.draw()
  File "D:\Development\Continuum\anaconda3\lib\site-packages\matplotlib\backends\backend_tkagg.py", line 12, in draw
    super(FigureCanvasTkAgg, self).draw()
  File "D:\Development\Continuum\anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 433, in draw
    self.figure.draw(s

KeyboardInterrupt: 

In [132]:
im=image_right
# This part will will calculate the X and Y offsets
bunchX=[]; bunchY=[]

tt = np.array([[1],[1],[1]])
tmp = np.dot(H,tt)
bunchX.append(tmp[0]/tmp[2])
bunchY.append(tmp[1]/tmp[2])

tt = np.array([[im.shape[1]],[1],[1]])
tmp = np.dot(H,tt)
bunchX.append(tmp[0]/tmp[2])
bunchY.append(tmp[1]/tmp[2])

tt = np.array([[1],[im.shape[0]],[1]])
tmp = np.dot(H,tt)
bunchX.append(tmp[0]/tmp[2])
bunchY.append(tmp[1]/tmp[2])

tt = np.array([[im.shape[1]],[im.shape[0]],[1]])
tmp = np.dot(H,tt)
bunchX.append(tmp[0]/tmp[2])
bunchY.append(tmp[1]/tmp[2])

refX1 = int(np.min(bunchX))
refX2 = int(np.max(bunchX))
refY1 = int(np.min(bunchY))
refY2 = int(np.max(bunchY))

# Final image whose size is defined by the offsets previously calculated
final = np.zeros((int(refY2-refY1),int(refX2-refX1),3))
# final = np.zeros((2000,2000,3))

# Iterate over the imagine to transform every pixel
for i in range(im.shape[0]):
        for j in range(im.shape[1]):

                tt = np.array([[j],[i],[1]])
                tmp = np.dot(H,tt)
                x1=int(tmp[0]/tmp[2])-refX1
                y1=int(tmp[1]/tmp[2])-refY1

                if x1>0 and y1>0 and y1<refY2-refY1 and x1<refX2-refX1:
                        final[y1,x1,:]=im[i,j,:]
# Hi = np.linalg.inv(H)
# for i in range(final.shape[0]):
#         for j in range(final.shape[1]):
#                 if sum(final[i,j,:])==0:
#                         tt = np.array([[j+refX1],[i+refY1],[1]])
#                         tmp = np.dot(Hi,tt)
#                         x1=int(tmp[0]/tmp[2])
#                         y1=int(tmp[1]/tmp[2])

#                         if x1>0 and y1>0 and x1<im.shape[1] and y1<im.shape[0]:
#                                 final[i,j,:] = im[y1,x1,:]


In [48]:
(tmp/tmp[2]).astype(int)

array([[1037],
       [ 456],
       [   1]])

In [79]:
np.zeros((int(refY2-refY1),int(refX2-refX1),3)).shape


(1689, 2740, 3)

In [133]:
plt.figure(figsize=(20,20))
cv2.imshow("warp",final.astype(np.uint8))


In [87]:
a=image_left
print ("Homography is : ", H)
xh = np.linalg.inv(H)
print ("Inverse Homography :", xh)
# start_p is denoted by f1
f1 = np.dot(xh, np.array([0,0,1]))
f1 = f1/f1[-1]
# transforming the matrix 
xh[0][-1] += abs(f1[0])
xh[1][-1] += abs(f1[1])
ds = np.dot(xh, np.array([a.shape[1], a.shape[0], 1]))
offsety = abs(int(f1[1]))
offsetx = abs(int(f1[0]))
# dimension of warped image
dsize = (int(ds[0])+offsetx, int(ds[1]) + offsety)
print ("image dsize =>", dsize)
tmp = cv2.warpPerspective(a, H, dsize)
cv2.imshow("warped", tmp)
# cv2.waitKey()
tmp[offsety:b.shape[0]+offsety, offsetx:b.shape[1]+offsetx] = b
a = tmp


Homography is :  [[-9.85055676e-01 -2.74229971e-01  5.52053203e+02]
 [-9.65955728e-01 -1.67145082e+00  1.31712427e+03]
 [-1.24705458e-03 -1.23327803e-03  1.00000000e+00]]
Inverse Homography : [[ 1.80253051e-01  1.55706817e+00 -2.15036154e+03]
 [ 2.59088179e+00  1.13586885e+00 -2.92638502e+03]
 [ 3.42006299e-03  3.34259108e-03 -5.29066457e+00]]
image dsize => (388, 2586)


ValueError: could not broadcast input array from shape (3,3) into shape (3,0,3)

In [ ]:
np.unique(final)

In [15]:
np.min(np.dot(im,H))

-654.2411261813959

In [9]:
final.shape

(12, 240, 3)

In [39]:
np.unique(im)

ERROR! Session/line number was not unique in database. History logging moved to new session 1337


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
np.dot(im,H)

In [63]:

H

(array([[ 1.88912702e-01, -1.21982256e-01,  3.80739984e+02],
        [-9.54645115e-01,  1.08769673e+00,  6.92690398e+02],
        [-1.02313781e-03,  1.21423021e-03,  1.00000000e+00]]), array([[1],
        [1],
        [1],
        [1],
        [1]], dtype=uint8))

In [66]:
image_right.shape

(960, 1280, 3)

In [142]:
plt.figure(figsize=(20,20))
# cv2.imshow("warped",cv2.warpPerspective(image_right,h, (image_right.shape[1],(image_right.shape[0]+image_left.shape[0]))))
final=cv2.warpPerspective(image_right,h_cv2, (image_right.shape[1],(image_right.shape[0]+image_left.shape[0])))

In [108]:
warp=cv2.warpPerspective(image_right,H, (image_right.shape[1],(image_right.shape[0]+image_left.shape[0])))
cv2.imshow("falanfian",warp)

In [125]:
final=np.dot(np.dot(np.linalg.inv(t2),H),t1).astype(np.uint8)

In [144]:
final_res_img.shape

(1920, 1280, 3)

In [145]:
final_res_img=np.zeros((max(image_left.shape[0],final.shape[0]),max(image_left.shape[1],final.shape[1]),3),dtype=np.uint8)
for j in range(final_res_img.shape[0]):
    for i in range(final_res_img.shape[1]):
        if j<image_left.shape[0] and i<image_left.shape[1]:
            if np.sum(image_left[j,i,:])>np.sum(final[j,i,:]):                                             
                final_res_img[j,i]=image_left[j,i]
            else:
                final_res_img[j,i]=final[j,i]
                                                

In [115]:
final_res_img[0:image_left.shape[0],0:image_left.shape[1]]=image_left
final_res_img[image_left.shape[0]:,image_left.shape[1]:]=final

ValueError: could not broadcast input array from shape (1252,1852,3) into shape (292,572,3)

In [78]:
cv2.imshow("warp2",np.)

In [129]:
cv2.imshow("warp2",final)

In [40]:
res=get_stitched_image(imageA,imageB,M)
plt.figure(figsize=(20,20))
plt.imshow(res)

plt.show()

NameError: name 'get_stitched_image' is not defined